# Challenge 3

In this challenge we will work on the `Orders` data set. In your work you will apply the thinking process and workflow we showed you in Challenge 2.

You are serving as a Business Intelligence Analyst at the headquarter of an international fashion goods chain store. Your boss today asked you to do two things for her:

**First, identify two groups of customers from the data set.** The first group is **VIP Customers** whose **aggregated expenses** at your global chain stores are **above the 95th percentile** (aka. 0.95 quantile). The second group is **Preferred Customers** whose **aggregated expenses** are **between the 75th and 95th percentile**.

**Second, identify which country has the most of your VIP customers, and which country has the most of your VIP+Preferred Customers combined.**

## Q1: How to identify VIP & Preferred Customers?

We start by importing all the required libraries:

In [1]:
# import required libraries
import numpy as np
import pandas as pd

Next, extract and import `Orders` dataset into a dataframe variable called `orders`. Print the head of `orders` to overview the data:

In [2]:
ls

Orders.csv         Pokemon.csv        challenge-2.ipynb
Orders.zip         challenge-1.ipynb  challenge-3.ipynb


In [3]:
# your code here
orders = pd.read_csv('Orders.csv')
orders

,Unnamed: 0,InvoiceNo,StockCode,year,month,day,hour,Description,Quantity,InvoiceDate,UnitPrice,CustomerID,Country,amount_spent
0,0,536365,85123A,2010,12,3,8,white hanging heart t-light holder,6,2010-12-01 08:26:00,2.55,17850,United Kingdom,15.30
1,1,536365,71053,2010,12,3,8,white metal lantern,6,2010-12-01 08:26:00,3.39,17850,United Kingdom,20.34
2,2,536365,84406B,2010,12,3,8,cream cupid hearts coat hanger,8,2010-12-01 08:26:00,2.75,17850,United Kingdom,22.00
3,3,536365,84029G,2010,12,3,8,knitted union flag hot water bottle,6,2010-12-01 08:26:00,3.39,17850,United Kingdom,20.34
4,4,536365,84029E,2010,12,3,8,red woolly hottie white heart.,6,2010-12-01 08:26:00,3.39,17850,United Kingdom,20.34
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
397919,541904,581587,22613,2011,12,5,12,pack of 20 spaceboy napkins,12,2011-12-09 12:50:00,0.85,12680,France,10.20
397920,541905,581587,22899,2011,12,5,12,children's apron dolly girl,6,2011-12-09 12:50:00,2.10,12680,France,12.60
397921,541906,581587,23254,2011,12,5,12,childrens cutlery dolly girl,4,2011-12-09 12:50:00,4.15,12680,France,16.60
397922,541907,581587,23255,2011,12,5,12,childrens cutlery circus parade,4,2011-12-09 12:50:00,4.15,12680,France,16.60


---

"Identify VIP and Preferred Customers" is the non-technical goal of your boss. You need to translate that goal into technical languages that data analysts use:

## How to label customers whose aggregated `amount_spent` is in a given quantile range?


We break down the main problem into several sub problems:

#### Sub Problem 1: How to aggregate the  `amount_spent` for unique customers?

#### Sub Problem 2: How to select customers whose aggregated `amount_spent` is in a given quantile range?

#### Sub Problem 3: How to label selected customers as "VIP" or "Preferred"?

*Note: If you want to break down the main problem in a different way, please feel free to revise the sub problems above.*

Now in the workspace below, tackle each of the sub problems using the iterative problem solving workflow. Insert cells as necessary to write your codes and explain your steps.

Sub Problem 1: How to aggregate the amount_spent for unique customers?

In [17]:
orders.duplicated()

0         False
1         False
2         False
3         False
4         False
          ...  
397919    False
397920    False
397921    False
397922    False
397923    False
Length: 397924, dtype: bool

In [19]:
orders = orders.drop_duplicates()

In [20]:
orders.head()

,Unnamed: 0,InvoiceNo,StockCode,year,month,day,hour,Description,Quantity,InvoiceDate,UnitPrice,CustomerID,Country,amount_spent,percentile,percent
0,0,536365,85123A,2010,12,3,8,white hanging heart t-light holder,6,2010-12-01 08:26:00,2.55,17850,United Kingdom,15.30,0.000002,0.000172
1,1,536365,71053,2010,12,3,8,white metal lantern,6,2010-12-01 08:26:00,3.39,17850,United Kingdom,20.34,0.000002,0.000228
2,2,536365,84406B,2010,12,3,8,cream cupid hearts coat hanger,8,2010-12-01 08:26:00,2.75,17850,United Kingdom,22.00,0.000002,0.000247
3,3,536365,84029G,2010,12,3,8,knitted union flag hot water bottle,6,2010-12-01 08:26:00,3.39,17850,United Kingdom,20.34,0.000002,0.000228
4,4,536365,84029E,2010,12,3,8,red woolly hottie white heart.,6,2010-12-01 08:26:00,3.39,17850,United Kingdom,20.34,0.000002,0.000228


In [23]:
unique_customers = orders.groupby(['CustomerID']).agg({'amount_spent':np.sum}).reset_index()
unique_customers

,CustomerID,amount_spent
0,12346,77183.60
1,12347,4310.00
2,12348,1797.24
3,12349,1757.55
4,12350,334.40
...,...,...
4334,18280,180.60
4335,18281,80.82
4336,18282,178.05
4337,18283,2094.88


Sub Problem 2: How to select customers whose aggregated amount_spent is in a given quantile range?¶

In [31]:
unique_customers.describe().T

,count,mean,std,min,25%,50%,75%,max
CustomerID,4339.0,15299.936852,1721.889758,12346.0,13812.500,15299.00,16778.50,18287.00
amount_spent,4339.0,2053.793018,8988.248381,0.0,307.245,674.45,1661.64,280206.02


In [29]:
def percentile(x):
    if (x > 280206.02):
        return 0.95
    elif (x > 1661.64) & (x<280206.01): 
        return 0.75
    else:
        return 'not VIP'

percentile(280206.03)

0.95

In [40]:
# we have an option already
list(map(percentile,unique_customers['amount_spent']))

[0.75,
 0.75,
 0.75,
 0.75,
 'not VIP',
 0.75,
 'not VIP',
 'not VIP',
 'not VIP',
 0.75,
 0.75,
 'not VIP',
 0.75,
 0.75,
 'not VIP',
 0.75,
 'not VIP',
 'not VIP',
 'not VIP',
 'not VIP',
 0.75,
 0.75,
 'not VIP',
 'not VIP',
 'not VIP',
 'not VIP',
 'not VIP',
 0.75,
 'not VIP',
 0.75,
 0.75,
 0.75,
 'not VIP',
 'not VIP',
 0.75,
 'not VIP',
 'not VIP',
 'not VIP',
 'not VIP',
 0.75,
 0.75,
 'not VIP',
 'not VIP',
 'not VIP',
 'not VIP',
 'not VIP',
 0.75,
 0.75,
 0.75,
 0.75,
 0.75,
 'not VIP',
 'not VIP',
 'not VIP',
 'not VIP',
 0.75,
 0.75,
 'not VIP',
 'not VIP',
 'not VIP',
 'not VIP',
 0.75,
 0.75,
 'not VIP',
 'not VIP',
 'not VIP',
 0.75,
 0.75,
 'not VIP',
 0.75,
 0.75,
 0.75,
 'not VIP',
 0.75,
 'not VIP',
 0.75,
 0.75,
 'not VIP',
 'not VIP',
 0.75,
 'not VIP',
 'not VIP',
 'not VIP',
 'not VIP',
 0.75,
 'not VIP',
 0.75,
 'not VIP',
 'not VIP',
 0.75,
 0.75,
 0.75,
 0.75,
 'not VIP',
 'not VIP',
 'not VIP',
 'not VIP',
 'not VIP',
 'not VIP',
 'not VIP',
 0.75,
 0.75,
 

Sub Problem 3: How to label selected customers as "VIP" or "Preferred"?

In [38]:
def label_customer(x):
    if (x == 0.95):
        return "VIP"
    elif (x == 0.75): 
        return "Preferred"
    else:
        return 'not label'

label_customer(0.85)

'not label'

In [41]:
unique_customers['label'] = unique_customers['amount_spent'].apply(label_customer)

In [42]:
unique_customers

,CustomerID,amount_spent,label
0,12346,77183.60,not label
1,12347,4310.00,not label
2,12348,1797.24,not label
3,12349,1757.55,not label
4,12350,334.40,not label
...,...,...,...
4334,18280,180.60,not label
4335,18281,80.82,not label
4336,18282,178.05,not label
4337,18283,2094.88,not label


Now we'll leave it to you to solve Q2 & Q3, which you can leverage from your solution for Q1:

## Q2: How to identify which country has the most VIP Customers?

In [46]:
# your code here
customers_country = unique_customers.merge(orders[['Country','CustomerID']],
                                how = 'inner',
                                left_on = 'CustomerID',
                                right_on = 'CustomerID')

customers_country.drop_duplicates(inplace=True)
customers_country.reset_index(inplace=True)
customers_country

,index,CustomerID,amount_spent,label,Country
0,0,12346,77183.60,not label,United Kingdom
1,1,12347,4310.00,not label,Iceland
2,183,12348,1797.24,not label,Finland
3,214,12349,1757.55,not label,Italy
4,287,12350,334.40,not label,Norway
...,...,...,...,...,...
4342,397069,18280,180.60,not label,United Kingdom
4343,397079,18281,80.82,not label,United Kingdom
4344,397086,18282,178.05,not label,United Kingdom
4345,397098,18283,2094.88,not label,United Kingdom


## Q3: How to identify which country has the most VIP+Preferred Customers combined?

In [50]:
# your code here
